In [1]:
import pandas as pd
import spacy
from spacy import displacy
from tqdm import tqdm
import json
from tqdm import tqdm
import spacy

In [2]:
# Load spaCy model for tokenization
nlp = spacy.load('de_core_news_lg')

In [3]:
# Load testing data
with open('test_data.json', 'r', encoding='utf-8') as f:
    testing_data = json.load(f)

In [4]:
# Load job titles dictionary
with open('positions_list_HISCO.txt', 'r', encoding='ANSI') as f:
    positions_dict = [line.strip() for line in f.readlines()]

In [5]:
testing_data[0]

{'text': 'Gute Köchin neben Stuben mädchen gesucht. 2. Bez., Unt. Donaustr. 29, Tür 11. 21856',
 'entities': [[5, 11, 'position']]}

In [6]:
positions_dict[0]

'Abdecker'

In [7]:
positions = []
exact_matches = ['Mädchen für Alles', 'Mädchen für alles', 'Stütze der Hausfrau']

for entry in tqdm(testing_data):
    positions_row = []
    text = entry['text'].replace('ſ', 's')
    doc = nlp(text)
    
    for exact_match in exact_matches:
        if exact_match in text:
            positions_row.append(exact_match)

    for token in doc:
        for position_name in positions_dict:
            if position_name.lower() in token.text.lower():
                positions_row.append(token.text)

    positions.append(positions_row)

100%|████████████████████████████████████████████████████████████████████████████████| 637/637 [00:14<00:00, 42.81it/s]


In [8]:
print(positions)

[['Köchin'], ['Lehrjunge'], ['Krankenschwestern', 'Angebote'], ['Hebamme', 'Krankenpflegerin', 'Krankenpflegerin', 'Krankenpflegerin', 'Krankenpflegerin', 'Krankenpflegerin'], ['Gefangenaufsehersstelle'], ['Gärtner', 'Gärtner', 'Gärtner', 'Gärtner'], ['Lehrer', 'Lehrerinnen⸗', 'Lehrerinnen⸗'], ['Wagenschmied', 'Schmiedmstr'], ['Zuschneider'], ['Meierin', 'landwirtschaft', 'landwirtschaft', 'Wirtschastsführung', 'Kochen', 'Meierin'], ['Kaufmann', 'Bäckerei'], ['Servierkellnerin'], ['Straßenmeisterstelle', 'Straßenmeisterstelle', 'provisorischer', 'Straßenmeisters', 'Straßenmeisters', 'Unteroffizieren', 'Unteroffizieren'], ['Jackennäherinnen', 'Jackennäherinnen'], ['Hausadministrator', 'Hausadministrator', 'Staatsbeamter'], ['Schauspieler'], ['Maschinnäherinnen', 'Maschinnäherinnen', 'Knopflochnäherinnen', 'Knopflochnäherinnen'], ['Handelsangestellter', 'Handelsangestellter'], ['Hausknecht', 'Hausknecht'], ['Lehrling'], [], [], ['Miedernäherinnen', 'Miedernäherinnen'], [], ['Mädchen für 

In [9]:
total_annotations = 0
correct_matches = 0

for entry, identified in zip(testing_data, positions):
    annotated_positions = [entry['text'][start:end] for start, end, _ in entry['entities']]
    total_annotations += len(annotated_positions)
    
    identified_set = set(identified)
    annotated_set = set(annotated_positions)
    
    # Calculate the number of correct matches
    matches = len(identified_set & annotated_set)
    
    # Calculate the fraction of correct matches compared to total annotated entities
    match_fraction = matches / len(annotated_positions) if len(annotated_positions) > 0 else 0
    
    correct_matches += match_fraction
    
    # Print annotations and predictions for mismatches
    if match_fraction != 1:
        print(f"Annotations: {annotated_positions}")
        print(f"Predictions: {identified}")
        print()

Annotations: ['Lehrstellen']
Predictions: ['Lehrer', 'Lehrerinnen⸗', 'Lehrerinnen⸗']

Annotations: ['Cassier', 'Rechnungsführer', 'Hausadministrator', 'Staatsbeamter']
Predictions: ['Hausadministrator', 'Hausadministrator', 'Staatsbeamter']

Annotations: ['Maschinnäherinnen', 'Vorrichterinnen', 'Knopflochnäherinnen']
Predictions: ['Maschinnäherinnen', 'Maschinnäherinnen', 'Knopflochnäherinnen', 'Knopflochnäherinnen']

Annotations: ['Wäscherin']
Predictions: []

Annotations: ['Supplentenstelle']
Predictions: []

Annotations: ['Schreibstelle']
Predictions: []

Annotations: ['Kinderfräulein']
Predictions: ['Stütze']

Annotations: ['Kontrolorspostens', 'Amtsschreiberspostens']
Predictions: ['Amtsschreiberspostens', 'Amtsschreiberspostens', 'Verwalter']

Annotations: ['Bonne', 'Haushälterin']
Predictions: ['Haushälterin']

Annotations: ['Maler', 'Anstreicher', 'Lackirer']
Predictions: ['Maler', 'Anstreicher']

Annotations: ['Stallbursch']
Predictions: []

Annotations: ['Kommis']
Predictions

In [10]:
accuracy = correct_matches / len(testing_data) if testing_data else 0

print(f'Accuracy: {accuracy:.2%}')

Accuracy: 68.45%
